In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
suppressMessages(suppressWarnings(library(pheatmap)))
show_env()

You are in Singularity: singularity_proj_combeffect 
BASE DIRECTORY:     /data/reddylab/Kuei 
WORK DIRECTORY:     /data/reddylab/Kuei/out 
CODE DIRECTORY:     /data/reddylab/Kuei/code 
PATH OF SOURCE:     /data/reddylab/Kuei/source 
PATH OF EXECUTABLE: /data/reddylab/Kuei/bin 
PATH OF ANNOTATION: /data/reddylab/Kuei/annotation 
PATH OF PROJECT:    /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS:    /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 


In [2]:
ASSAY    = "Tewhey_K562_TileMPRA"
FOLDER   = "coverage_astarrseq_peak_macs_input"
DATASETS = c("OL13", "OL43", "OL45")
TYPE     = "raw"

In [3]:
###
lst = lapply(DATASETS, function(DATASET){
    fdiry = file.path(FD_RES, "results", ASSAY, FOLDER, "summary")
    fname = paste("result", "Log2FC", "raw", "deseq", DATASET, "tsv", sep=".")
    fpath = file.path(fdiry, fname)
    dat   = read_tsv(fpath, show_col_types = FALSE)
    dat   = dat %>% dplyr::mutate(Dataset = DATASET)
    return(dat)
})

###
dat = bind_rows(lst)

###
dat_log2fc = dat
print(dim(dat))
head(dat)

[1] 1451    8


Peak,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Dataset
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
chr11_61792068_61793464,260242.12,1.6665705,0.04117109,40.479145,0.000000e+00,0.000000e+00,OL13
chr11_61800085_61801113,216437.58,1.8554055,0.04972890,37.310405,1.112851e-291,6.120682e-293,OL13
chr11_61806630_61807154,49327.33,-1.3562455,0.08825841,-15.366758,2.735191e-53,4.628784e-53,OL13
chr11_61814735_61817343,682819.52,2.5310135,0.03452518,73.309201,0.000000e+00,0.000000e+00,OL13
chr11_61822094_61822443,20363.10,-1.7731145,0.14597377,-12.146802,5.965339e-34,7.719850e-34,OL13
chr11_61825795_61826306,29539.40,-0.3921839,0.07941651,-4.938318,7.879932e-07,7.879932e-07,OL13


In [61]:
get_label = function(x_chrom, x_start, x_end){
    ### init
    txt     = "Unlabeled"
    x_start = as.integer(x_start)
    x_end   = as.integer(x_end)
    
    ### Region: chr2:59,553,301-61,553,700
    ### Label:  BCL11A
    if (x_chrom == "chr2"){
        if ((x_start >= 59553301) && (x_end <= 61553700)){
            txt = "BCL11A"
        }
    }
    
    ### Region: chr8:126,735,901-128,736,550
    ### Label: MYC
    if (x_chrom == "chr8"){
        if ((x_start >= 126735901) && (x_end <= 128736550)){
            txt = "MYC"
        }
    }
    
    ### Region: chr11:4,505,501-6,505,900
    ### Label: HBE1
    if (x_chrom == "chr11"){
        if ((x_start >= 4505501) && (x_end <= 34870100)){
            txt = "HBE1"
        }
    }
    
    ### Region: chr11:32,869,701-34,870,100
    ### Label: LMO2
    if (x_chrom == "chr11"){
        if ((x_start >= 4505501) && (x_end <= 34870100)){
            txt = "LMO2"
        }
    }
                                 #61,897,731 61,898,477
    ### Region: chr11:61,787,329-61,898,348
    ### Label:  FADS
    if (x_chrom == "chr11"){
        if ((x_start >= 61787329) && (x_end <= 61898348)){
            txt = "FADS"
        }
    }
    
    ### Region: chr16:10,001-1,173,100
    ### Label:  HBA2
    if (x_chrom == "chr16"){
        if ((x_start >= 10001) && (x_end <= 1173100)){
            txt = "HBA2"
        }
    }
    
    ### Region: chr20:56,391,301-58,391,700
    ### Label:  RBM38
    if (x_chrom == "chr20"){
        if ((x_start >= 56391301) && (x_end <= 58391700)){
            txt = "RBM38"
        }
    }
    
    ### Region: chrX:47,785,501-49,880,650
    ### Label:  GATA1
    if (x_chrom == "chrX"){
        if ((x_start >= 47785501) && (x_end <= 49880650)){
            txt = "GATA1"
        }
    }
    return(txt)
}

fun_label_peak = function(xs_chrom, xs_start, xs_end, xs_dataset){
    xs_label = sapply(seq_along(xs_chrom), function(idx){
        x_chrom   = xs_chrom[idx]
        x_start   = xs_start[idx]
        x_end     = xs_end[idx]
        x_dataset = xs_dataset[idx]
        x_label   = get_label(x_chrom, x_start, x_end)
        
        if (x_dataset == "OL13"){
            return("FADS")
        } else {
            return(x_label)
        }
    })
    return(xs_label)
}

In [62]:
dat = dat_log2fc
dat = dat %>% 
    dplyr::select(Peak, Dataset) %>%
    tidyr::separate(Peak, into=c("Chrom", "Start", "End"), remove=FALSE) %>%
    dplyr::mutate(Region = fun_label_peak(Chrom, Start, End, Dataset))

#dat$Label = fun_label_peak(dat$Chrom, dat$Start, dat$End)
dat_peak = dat
print(dim(dat))
head(dat)

[1] 1451    6


Peak,Chrom,Start,End,Dataset,Region
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
chr11_61792068_61793464,chr11,61792068,61793464,OL13,FADS
chr11_61800085_61801113,chr11,61800085,61801113,OL13,FADS
chr11_61806630_61807154,chr11,61806630,61807154,OL13,FADS
chr11_61814735_61817343,chr11,61814735,61817343,OL13,FADS
chr11_61822094_61822443,chr11,61822094,61822443,OL13,FADS
chr11_61825795_61826306,chr11,61825795,61826306,OL13,FADS


In [63]:
dat = dat_peak
table(dat$Chrom)


chr11 chr16  chr2  chr8  chrX 
  589   284   184   188   206 

In [65]:
dat = dat_peak
table(dat$Chrom, dat$Region)

       
        BCL11A FADS GATA1 HBA2 LMO2 MYC
  chr11      0   22     0    0  567   0
  chr16      0    0     0  284    0   0
  chr2     184    0     0    0    0   0
  chr8       0    0     0    0    0 188
  chrX       0    0   206    0    0   0

In [67]:
dat %>% dplyr::filter(Region == "Unlabeled")

Peak,Chrom,Start,End,Dataset,Region
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


In [68]:
fdiry = file.path(FD_RES, "results", "comparison")
dir(fdiry)

[1] "comparison_global"  "comparison_local"   "dat_peaks_info.tsv"

In [69]:
fdiry = file.path(FD_RES, "results", "comparison")
fname = "dat_peaks_info.tsv"
fpath = file.path(fdiry, fname)

write_tsv(dat_peak, fpath)